# Рекомендация тарифов

Для пользователей тарифов из проекта "Определение перспективного тарифа" необходимо создать модель классификации, которая подберет подходящий тариф.

Значение accuracy должно бьть не менее 0,75.

<div class="alert alert-info">Импортируем все, что нам нужно прочитаем файл
</div>

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [ ]:

display(df)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


<div class="alert alert-info">признаки - все, кроме столбца is_ultra, соответственно цель - is_ultra
</div>

In [ ]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra'] # < напишите код здесь >

print(features.shape)
print(target.shape)

(3214, 4)
(3214,)


In [ ]:
print(df.duplicated().sum())

0


In [ ]:
#делим данные на выборки
df, df_test = train_test_split(df, test_size=0.2, train_size=0.8)
df_train, df_valid = train_test_split(df, test_size=0.25, train_size=0.75)

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 781 to 512
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2509 to 2205
Data columns (total 5 columns):
calls       1928 non-null float64
minutes     1928 non-null float64
messages    1928 non-null float64
mb_used     1928 non-null float64
is_ultra    1928 non-null int64
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [ ]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2027 to 1011
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [ ]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

обучим разные модели

In [ ]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    # < обучаем модель >
    if result > best_result:
        best_model = model.fit(features_train, target_train) #сохраняем наилучшую модель
        best_result = model.score(features_valid, target_valid)
    predictions_valid = model.predict(features_valid) # < находим предсказания на валидационной выборке >
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.8195956454121306


In [ ]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучаем модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # считаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model.fit(features_train, target_train) #сохраняем наилучшую модель
        best_result = model.score(features_valid, target_valid) #сохраняем наилучшее значение accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.8164852255054432


In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
model = LogisticRegression(random_state=12345)  #модель логистической регрессии с параметром random_state=12345
model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
result = model.score(features_valid, target_valid) # получаем метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7013996889580093


Проверим модель на тестовой выборке

In [ ]:
best_model = None
best_result = 0
for est in range(1, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    # < обучаем модель >

    predictions_test = model.predict(features_test) # < находим предсказания на тестовой выборке >
    if result > best_result:
        best_model = model.fit(features_train, target_train) #сохраняем наилучшую модель
        best_result = model.score(features_test, target_test) #сохраняем наилучшее значение accuracy на тестовых данных

print("Accuracy наилучшей модели на тестовой выборке:", best_result)
    

Accuracy наилучшей модели на тестовой выборке: 0.7667185069984448


In [ ]:
best_model = None
best_result = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    # < обучаем модель >
    predictions_test = model.predict(features_test) # < находим предсказания на тестовой выборке >
    if result > best_result:
        best_model = model.fit(features_train, target_train) #сохраняем наилучшую модель
        best_result = model.score(features_test, target_test) #сохраняем наилучшее значение accuracy на тестовых данных

print("Accuracy наилучшей модели на тестовой выборке:", best_result)
    

Accuracy наилучшей модели на тестовой выборке: 0.7838258164852255


<div class="alert alert-info"> DecisionTreeClassifier и RandomForestClassifier показали лучшие результаты; на тестовой выборке лучше выглядит DecisionTreeClassifier с 0,78
</div>